In [1]:
import pandas as pd

df = pd.read_csv('data/2019-12.csv', encoding='cp949', low_memory=False)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

use_category = [1001, 1003, 1004, 1007, 1008, 1010, 1099, 1102, 1104, 1105, 1199,
                1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1299, 
                3001, 3002, 3101, 3102, 3201, 3199, 3202, 3203,
                3299, 6101, 6102, 6103, 6109, 6110, 5502, 5605, 
                6001, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010, 6011, 6013,
                6016, 6021, 6099, 6019, 6020, 6018, 6107, 6113, 6201, 6202,
                6203, 6204, 6205, 6206, 6207, 6208, 6111, 6112, 6299, 
                7004, 7006, 7007, 7008, 7010, 7099, 7101, 7103,
                7106, 7108, 7199, 8108, 8115, 8109, 8110, 8101, 8102,
                8106, 8112, 8113, 8114, 8111, 6114, 6115, 6116, 6105, 6199, 8406]


age_mask = df['연령'] != '0.기타'
gender_mask = (df['성별'] != '0.기타') & (df['성별'] != '3.기업')
store_mask = df['가맹점업종코드'].isin(use_category)
df = df[age_mask & gender_mask & store_mask]

group_data = (df.groupby(['성별', '연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수']
              .sum()
              .reset_index())

gender_data = group_data['성별'].apply(lambda x: int(x[0]))
age_data = group_data['연령'].apply(lambda x: int(x[0]))
store_data = group_data['가맹점업종코드']
cash_avg_data = group_data['결제금액'] // group_data['결제건수']
count_data = group_data['결제건수']

agg_data = pd.DataFrame({
    'gender': gender_data,
    'age': age_data,
    'store': store_data,
    'cash': cash_avg_data,
    'count': count_data})

agg_data['perc_count'] = (agg_data.groupby(['gender', 'age'])
                          .apply(lambda g: g / g.sum())['count'] * 100)
print(agg_data)


# test_df = df[store_mask]
# test2_df = test_df.groupby('가맹점업종코드')['결제건수'].sum().reset_index()
# test_dict = test2_df.set_index('가맹점업종코드').to_dict()
# agg_data['전체 결제건수'] = agg_data['store'].apply(lambda x: test_dict['결제건수'][x])
# agg_data['테스트'] = agg_data['count'] / agg_data['전체 결제건수'] * 100
# print(agg_data)

     age    cash  count  gender  store  perc_count
0      1   44393   1028       1   1004    0.252559
1      1  149042     14       1   1010    0.003440
2      1   49700      3       1   1099    0.000737
3      1   15088     66       1   1201    0.016215
4      1   12000      3       1   1204    0.000737
..   ...     ...    ...     ...    ...         ...
731    7  139729    424       2   7106   11.468758
732    7   42250     20       2   8101    0.540979
733    7    7000      3       2   8113    0.081147
734    7   30653    253       2   8114    6.843387
735    7   13535      7       2   8406    0.189343

[736 rows x 6 columns]


In [3]:
pd.set_option('display.max_row', 500)
category_data = pd.read_csv('data/category.csv').set_index('code').to_dict()
gender_data = {1: '남성', 2 :'여성'}
def change_age(n):
    return str(n) + '0대'

sort_by_count_values = (agg_data.groupby(['gender', 'age'])['store', 'perc_count']
                       .apply(lambda x: x.sort_values(by='perc_count', ascending=False)[:5])
                       .reset_index()
                       .drop('level_2', axis=1))

sort_by_count_values['gender'] = sort_by_count_values['gender'].apply(lambda x: gender_data[x])
sort_by_count_values['age'] = sort_by_count_values['age'].apply(lambda x: change_age(x))
sort_by_count_values['업종'] = sort_by_count_values['store'].apply(lambda x: category_data['name'][x])

# print(sort_by_count_values.set_index(['gender', 'age', '업종'])['perc_count'])

In [5]:
pd.set_option('display.max_row', 500)
category_data = pd.read_csv('data/category.csv').set_index('code').to_dict()
gender_data = {1: '남성', 2 :'여성'}
def change_age(n):
    return str(n) + '0대'

sort_by_count_values = (agg_data.groupby(['gender', 'age'])['store', 'cash', 'count', 'perc_count']
                       .apply(lambda x: x.sort_values(by='perc_count', ascending=False)[:5])
                       .reset_index()
                       .drop('level_2', axis=1))
sort_by_count_values['gender'] = sort_by_count_values['gender'].apply(lambda x: gender_data[x])
sort_by_count_values['age'] = sort_by_count_values['age'].apply(lambda x: change_age(x))
sort_by_count_values['업종'] = sort_by_count_values['store'].apply(lambda x: category_data['name'][x])
# sort_by_count_values['age'] = sort_by_count_values['count'].apply(lambda x: change_age(x))
# sort_by_count_values['설명'] = sort_by_count_values['store'].apply(lambda x: category_data['description'][x])

sort_by_cash_values = (agg_data.groupby(['gender', 'age'])['store', 'cash', 'count']
                       .apply(lambda x: x.sort_values(by='cash', ascending=False)[:5])
                      .reset_index()
                      .drop('level_2', axis=1))
sort_by_cash_values['업종'] = sort_by_cash_values['store'].apply(lambda x: category_data['name'][x])
# sort_by_cash_values['설명'] = sort_by_cash_values['store'].apply(lambda x: category_data['description'][x])

print(sort_by_count_values.set_index(['gender', 'age', '업종'])['perc_count'])
# print(sort_by_cash_values)

gender  age  업종           
남성      10대  비디오방/게임방         98.039225
             컴퓨터기기             0.509787
             문방구점              0.292360
             서점                0.273688
             기성복점              0.252559
        20대  비디오방/게임방         91.611631
             기성복점              1.715306
             화장품점              1.509553
             문방구점              1.361531
             완구점               1.055279
        30대  비디오방/게임방         41.697579
             기성복점              9.184733
             문방구점              6.722172
             서점                6.704586
             가전제품점             6.479268
        40대  서점               15.722498
             학습지              15.444971
             문방구점             10.210320
             기성복점              8.978073
             문리계학원             8.729321
        50대  서점               13.672061
             기성복점             10.821378
             차량 정비/부품/인테리어     9.842252
             문방구점              8.744135
             

In [ ]:
# male_df = df[df['성별'] == '1.남성']
# female_df = df[df['성별'] == '2.여성']
# male_group_data = male_df.groupby(['연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수'].sum().reset_index()
# female_group_data = female_df.groupby(['연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수'].sum().reset_index()

# print(male_group_data)
# male_age_data = male_group_data['연령'].apply(lambda x: int(x[0]))
# male_store_data = male_group_data['가맹점업종코드']
# male_cash_data = male_group_data['결제금액'] // male_group_data['결제건수']
# male_count_data = male_group_data['회원수']

# male_final_data = pd.DataFrame({
#     'age': male_age_data,
#     'store': male_store_data,
#     'cash': male_cash_data,
#     'count': male_count_data})

# print('남성 고객들이 특정 업종에서 평균적으로 사용하는 금액')
# male_cash = male_final_data.dropna().pivot(index='store', columns='age', values='cash').fillna(0)
# print(male_cash)


# print('10대 남성들이 특정 업종에서 평균적으로 사용하는 금액')
# print(male_cash[1].sort_values())
# print('-' * 100)

# print('남성 고객들이 특정 업종에서 평균적으로 결제하는 횟수')
# print(male_final_data.dropna().pivot(index='store', columns='age', values='count').fillna(0))

In [ ]:
# import numpy as np
# test_df = sort_by_count_values.set_index(['gender', 'age'])[['업종', 'perc_count']]
# test_df['perc_count'] = test_df['perc_count'].apply(lambda x: int(x*100) / 100)

# group = [('남성', '10대'), ('남성', '20대'), ('남성', '30대'), ('남성', '40대'), ('남성', '50대'), ('남성', '60대'), ('남성', '70대'),
#         ('여성', '10대'), ('여성', '20대'), ('여성', '30대'), ('여성', '40대'), ('여성', '50대'), ('여성', '60대'), ('여성', '70대')]

# for test in group:

#     fig, ax = plt.subplots(figsize=(6, 3), subplot_kw=dict(aspect="equal"))

#     data = test_df.loc[test]['perc_count']
    
#     anno = test_df.loc[test]['업종'].to_list()
#     anno.append('기타')

#     data_list = test_df.loc[test]['perc_count'].to_list()
#     data_list.append(int(100-data.sum())*100 / 100)

#     anno.pop(0)
#     data_list.pop(0)
#     wedges, texts = ax.pie(data_list, wedgeprops=dict(width=0.5), startangle=-40)

#     bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
#     kw = dict(arrowprops=dict(arrowstyle="-"),
#               bbox=bbox_props, zorder=0, va="center")

#     for i, p in enumerate(wedges):
#         ang = (p.theta2 - p.theta1)/2. + p.theta1
#         y = np.sin(np.deg2rad(ang))
#         x = np.cos(np.deg2rad(ang))
#         horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
#         connectionstyle = "angle,angleA=0,angleB={}".format(ang)
#         kw["arrowprops"].update({"connectionstyle": connectionstyle})
#         ax.annotate(anno[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
#                     horizontalalignment=horizontalalignment, **kw)

#     ax.set_title(test[0] + test[1])
# #    plt.savefig(test[0] + test[1] + '(3)')

#     plt.show()